In [1]:
import sys
import os
import nbimporter
import copy
import math
import numpy
import json
from collections import OrderedDict
import pprint

# 外皮入力情報から簡易熱負荷計算入力情報へのコンバート

#### Function

In [2]:
def convert_design_house(d):
    
    if d['InputLevel'] == 1:
        import LV1toLv2 as mf_1_to_2
        d2 = mf_1_to_2.convert(d)
                
    if d['InputLevel'] < 2:     
        import LV2toLv3 as mf_2_to_3
        d3 = mf_2_to_3.convert(d2) 
    elif d['InputLevel'] == 2:
        import LV2toLv3 as mf_2_to_3
        d3 = mf_2_to_3.convert(d)    
        
    if d['InputLevel'] < 3:     
        import LV3toLv4 as mf_3_to_4
        d4 = mf_3_to_4.convert(d3) 
    elif d['InputLevel'] == 3:
        import LV3toLv4 as mf_3_to_4
        d4 = mf_3_to_4.convert(d) 
        
    return d4

In [3]:
def convert_standard_house(d):
    
    import StandardHouse as mf_standard
    d_standard = mf_standard.convert(d) 

    return d_standard

In [4]:
def convert_input_for_calculation(d, HouseType):

    if d['InputLevel'] < 4:     
        d4 = convert_design_house(d)
    elif d['InputLevel'] == 4:
        d4 = cpoy.deepcopy(d) 
    
    import LV4toCalc as mf_4_to_calc
    d_calc = mf_4_to_calc.convert(d4) 
    
    with open('C:/Users/PC/Documents/e7_inputdataconvert/DesignHouse.json', 'w') as f_design:
        json.dump(d_calc, f_design, indent=4)

    d4['Common']['HouseType'] = HouseType
    with open('C:/Users/PC/Documents/e7_inputdataconvert/StandardHouse.json', 'w') as f_standard:
        json.dump(mf_4_to_calc.convert(convert_standard_house(d4)), f_standard, indent=4)

In [5]:
d = {
    'InputLevel': 1,
    'Region' : 6,
    'MainOccupantRoomFloorArea': 30.0,
    'OtherOccupantRoomFloorArea': 30.0,
    'TotalFloorArea': 90.0,
    'InsulationType' : 'FloorIns',
    'InsulationTypeBathroom' : 'NotInContactWithOutsideAir',
    'URoof' : 0.24,
    'UWall' : 0.53,
    'UFloor' : 0.48,
    'UFloorBathroom' : 0.48,
    'UDoor' : 4.65,
    'UW'    : 3.49,
    'EtaDC' : 0.51,       
    'EtaDH' : 0.51, 
    'IsInputFValue' : True, 
    'FValueC' : 0.9,
    'FValueH' : 0.7,
    'psi_entrance' : 1.8,
    'psi_bathroom' : 1.8
}

HouseType = 'detached'

convert_input_for_calculation(d, HouseType)

Importing Jupyter notebook from LV1toLv2.ipynb
Importing Jupyter notebook from RoundNumber.ipynb
Importing Jupyter notebook from LV2toLv3.ipynb
Importing Jupyter notebook from LV3toLv4.ipynb
Importing Jupyter notebook from LV4toCalc.ipynb
Importing Jupyter notebook from StandardHouse.ipynb
Importing Jupyter notebook from nu.ipynb
UA基準値0.87,ηAH基準値4.3,ηAC基準値2.8
U値{'Roof': 0.2049755366143829, 'Wall': 0.45265431002342893, 'Floor': 0.4099510732287648, 'Window': 3.971401021903669, 'EarthfloorPerimeter': 0.6490891992788776},開口部η0.4386836640077342,開口部ηH0.46743989470727487,開口部ηC0.40992743330819353
UA計算値0.8699999999999998,ηAH計算値4.3,ηAC計算値2.8000000000000003
